In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import cv2
import face_recognition

# import the necessary packages
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import argparse
import imutils
import dlib
import cv2

import matplotlib.pyplot as plt
%matplotlib inline

## 1.Alignment of Images:

In [2]:
pwd

'd:\\Ritu Projects\\Project 5 - BMI Regression'

In [3]:
# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor and the face aligner
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("req_models\shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=224)

In [4]:
def align_image(path_to_image,save_path):

    # load the input image, resize it, and convert it to grayscale
    image = cv2.imread(path_to_image)
    image = imutils.resize(image, width=800)
    
    try:
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    except:
        gray = cv2.imread(path_to_image,0)
    rects = detector(gray, 2)
    
    i = 0
    # loop over the face detections
    for rect in rects:
        # extract the ROI of the *original* face, then align the face
        # using facial landmarks
        (x, y, w, h) = rect_to_bb(rect)
        
        faceAligned = fa.align(image, gray, rect)
        
        
        print (faceAligned.shape)
        name = save_path.split('.')
        if (i == 0):
            cv2.imwrite(save_path ,faceAligned)
            cv2.waitKey(0)
            
        else:
            cv2.imwrite(name[0] + str(i) + '.' + name[1] ,faceAligned)
            cv2.waitKey(0)
        
        i += 1
        
    return True

## 2.Face Maskout:

In [5]:
def align_position(sample_img,r_start,r_stop,r_index,c_start,c_stop,c_index,flag):
    for i in range(r_start,r_stop,r_index):
        for j in range(c_start,c_stop,c_index):
            x,y=i,j
            if(flag!=1):
                temp=y
                y=x
                x=temp
            if(not np.array_equal(sample_img[x][y][:],np.array([128,128,128]))):
                sample_img[x][y][:]=[0,0,0]
            else:
                break
                
    return sample_img

In [6]:
def produce_face(sample_img):
    row=len(sample_img)
    column=len(sample_img[0])
    sample_img=align_position(sample_img,0,row,1,0,column,1,1)
    sample_img=align_position(sample_img,0,row,1,column-1,-1,-1,1)
    sample_img=align_position(sample_img,0,column,1,0,row,1,-1)
    sample_img=align_position(sample_img,0,column,1,row-1,-1,-1,-1)
    return sample_img

In [7]:
def connect_points(points1,points2,point_list):
    y1,x1 = points1
    y2,x2 = points2
    try:
        m=(y2-y1)/(x2-x1)
    except:
        m=(y2-y1)/1
    n=y1-m*x1
    point_list.append((x2,y2))
    if(abs(y2-y1)>abs(x2-x1)):
        for y in range(y1, y2, 1 if y2>y1 else -1):
            try:
                x=round((y-n)/m)
            except:
                x=n
            point_list.append((x,y))
    else:
        for x in range(x1,x2, 1 if x2>x1 else -1):
            y=round(m*x+n)
            point_list.append((x,y))
            
    return point_list

In [8]:
def remove_bound(point_list,i,j):
    lst=[]
    for (x,y) in point_list:
        if(x>=i):
            a=i-2
            b=y
        if(y>=j):
            b=j-2
            a=x
        if(x<i and y<j):
            a,b=x,y
        lst.append((a,b))
    return lst

In [9]:
def face_positions(image_name,save_dir):
    image = face_recognition.load_image_file(image_name)
    face_landmarks_list = face_recognition.face_landmarks(image)
    
    sample_img=np.array(cv2.imread(image_name))
    point_list=[]
    #print face_landmarks_list
    for data in face_landmarks_list[0]:
        if data in ['chin','left_eyebrow','right_eyebrow']:
            points=face_landmarks_list[0][data]
            points=remove_bound(points,len(sample_img[1]),len(sample_img[0]))
            for ln in range(len(points)-1):
                point_list=connect_points(points[ln],points[ln+1],point_list)
    
    point_list=connect_points(face_landmarks_list[0]['chin'][0],face_landmarks_list[0]['left_eyebrow'][0],point_list)
    point_list=connect_points(face_landmarks_list[0]['chin'][16],face_landmarks_list[0]['right_eyebrow'][4],point_list)
    point_list=connect_points(face_landmarks_list[0]['left_eyebrow'][4],face_landmarks_list[0]['right_eyebrow'][0],point_list)

    
    point_list=remove_bound(point_list,len(sample_img[1]),len(sample_img[0]))

    #print (point_list)
    for (x,y) in point_list:
        sample_img[int(x)][int(y)][:]=[128,128,128]
        
    sample_img=produce_face(sample_img)
    
    cv2.imwrite(save_dir,sample_img)
    
    return True

## 3.Enhancement of Images:

In [10]:
from PIL import Image,ImageFilter, ImageEnhance 
import numpy as np 
import cv2 
from matplotlib import pyplot as plt 
%matplotlib inline

In [11]:
def image_enhance(image_path):
    image = Image.open(image_path)
    enhancer = ImageEnhance.Sharpness(image)
    enhancer= ImageEnhance.Contrast(image) 
    enhanced_im = enhancer.enhance(1.8)
    enhanced_im.save(image_path)
    return True

## Sharpening of Images:

In [13]:
def sharpen_image(image_path):
    image = cv2.imread(image_path,cv2.COLOR_BGR2RGB)
    image= cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    kernel_sharpening = np.array([[-1,-1,-1], 
                              [-1, 9,-1],
                              [-1,-1,-1]])
    sharpened = cv2.filter2D(image, -1, kernel_sharpening)
    
    print(image_path)
    plt.imsave(image_path , sharpened)
    return True

### Function to perform Preprocessing the image:¶


In [14]:
def preprocess(image_path):
    align_image(image_path,image_path)
    face_positions(image_path,image_path)
    #sharpen_images(image_path)
    image_enhance(image_path)
    return True